In [1]:
import os
import shutil


try:
  import google.colab
  os.system("git clone https://github.com/matt-nann/AuthenticCursor.git")
  try:
    shutil.copytree("AuthenticCursor/src", "src")
  except:
    shutil.rmtree("src")
    shutil.copytree("AuthenticCursor/src", "src")
  try:
    shutil.copy("AuthenticCursor/requirementsGAN.txt", "requirementsGAN.txt")
  except:
    shutil.rmtree("requirementsGAN.txt")
    shutil.copy("AuthenticCursor/requirementsGAN.txt", "requirementsGAN.txt")
  os.system("pip install -r requirementsGAN.txt")
  shutil.rmtree("AuthenticCursor")
except:
  ...

In [22]:
import os
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader, TensorDataset
from src.mouseGAN.dataProcessing import MouseGAN_Data
from src.mouseGAN.dataset import getDataloader

USE_FAKE_DATA = True
SAVE_FAKE_DATA = False
dataset = MouseGAN_Data(USE_FAKE_DATA=USE_FAKE_DATA, equal_length=True, lowerLimit=25, upperLimit=30)

SAMPLES = 10000
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False
IN_COLAB = True
if USE_FAKE_DATA:
    if IN_COLAB:
        dataset.createFakeWindMouseDataset(save=SAVE_FAKE_DATA, samples=SAMPLES,
                                        low_radius = 200, high_radius = 300,
                                        max_width = 200, min_width = 50,
                                        max_height = 100, min_height = 25,)
    else:
        dataset.loadFakeWindMouseData()
else:
    df_moves, df_trajectory = dataset.collectRawMouseTrajectories()

In [3]:
norm_input_trajectories, norm_buttonTargets = dataset.processMouseData(SHOW_ALL=False, samples=18000)

processed fake data 200  out of  2000
processed fake data 400  out of  2000
processed fake data 600  out of  2000
processed fake data 800  out of  2000
processed fake data 1000  out of  2000
processed fake data 1200  out of  2000
processed fake data 1400  out of  2000
processed fake data 1600  out of  2000
processed fake data 1800  out of  2000
Number of sequences: 1997


## verifying the mean trajectory is centered around zero (even class distribution)

In [4]:
averageMove = np.array(dataset.input_trajectories).mean(axis=0)
# averageMove = averageMove * dataset.std_traj + dataset.mean_traj
df_sequence = pd.DataFrame(averageMove, columns=['dx','dy'])
df_sequence['velocity'] = np.sqrt(df_sequence['dx']**2 + df_sequence['dy']**2) / dataset.FIXED_TIMESTEP
df_target = pd.DataFrame(np.array(dataset.buttonTargets).mean(axis=0), columns=[dataset.targetColumns])
sequence_id = 0
dataset.SHOW_ONE = True
dataset.SHOW_ALL = False
df_abs = dataset.convertToAbsolute(df_sequence, df_target)
fig = dataset.plotTrajectory(df_abs, df_target[['width','height','start_x','start_y']], sequence_id)

In [5]:
# df_cleanedSeq, buttonTarget = dataset.processMouseData(SHOW_ALL=False)
# df_abs = dataset.convertToAbsolute(df_cleanedSeq, buttonTarget)
# dataset.plotTrajectory(df_abs, buttonTarget, 0)
# dataloader = getDataloader(norm_input_trajectories, norm_buttonTargets, BATCH_SIZE)

# df_sequence, df_target, start_x, start_y,left, top = dataset.processMouseData(SHOW_ONE=True, num_sequences=0)
# sequence_id = 0
# dataset.SHOW_ONE = True
# df_abs = dataset.convertToAbsolute(df_sequence, df_target)
# dataset.plotTrajectory(df_abs, df_target[['width','height','start_x','start_y']], sequence_id)

In [6]:
BATCH_SIZE = 256
dataloader = getDataloader(norm_input_trajectories, norm_buttonTargets, BATCH_SIZE)

### checking the dataloader

In [7]:
import plotly.graph_objects as go

fig = go.Figure()
for i, data in enumerate(dataloader, 0): 
    _input_trajectories_padded, _buttonTargets, trajectoryLengths = data
    # print(_input_trajectories_padded[0])
    if i == 1:
        break
    for ii in range(len(_input_trajectories_padded)):
        df_sequence = pd.DataFrame(_input_trajectories_padded[ii] * dataset.std_traj + dataset.mean_traj, columns=['dx','dy'])
        df_sequence['velocity'] = np.sqrt(df_sequence['dx']**2 + df_sequence['dy']**2) / dataset.FIXED_TIMESTEP
        df_target = pd.DataFrame(_buttonTargets[ii] * dataset.std_button + dataset.mean_button, columns=[dataset.targetColumns])
        sequence_id = 0
        dataset.SHOW_ONE = True
        df_abs = dataset.convertToAbsolute(df_sequence, df_target)

        fig.add_trace(go.Scatter(x=df_abs['x'], y=df_abs['y'],
                mode='lines+markers',
                marker=dict(
                            size=5, 
                            # symbol= "arrow-bar-up", angleref="previous",
                            # size=15,
                            # color='grey',),
                            color=df_abs['velocity'], colorscale='Viridis', showscale=True, colorbar=dict(title="Velocity")),
                
                ))
fig.update_layout(
    width=800,
    height=800,)
fig.show()
# pass

In [21]:
from src.mouseGAN.models import MouseGAN, LR_SCHEDULERS, LOSS_FUNC
from dataclasses import dataclass, asdict
LOAD_PRETRAINED = False

num_epochs = 100
num_feats = norm_input_trajectories[0].shape[1]
MAX_GRAD_NORM = 1000
latent_dim = 100
num_target_feats = 4 # width, height, start_x, start_y
MAX_SEQ_LEN = norm_input_trajectories[0].shape[0]
numBatches = len(dataloader)

@dataclass
class LR_Scheduler_Params:
    ideal_loss: float = 0.5 # LSGAN
    # ideal_loss: float = 0 # WGAN-GP
    loss_min: float = 0.1 * ideal_loss
    loss_max: float = 0.1 * ideal_loss
    lr_shrinkMin: float = 0.1
    discLossDecay: float = 0.8
    lr_growthMax: float = 2.0
    cooldown: int = int(numBatches / 8)
lr_Scheduler_Params = LR_Scheduler_Params()

@dataclass
class Plateua_EMA_Params:
    factor: float = 0.5
    min_lr: float = 1e-9
    verbose: bool = True
    patience: int = numBatches
    cooldown: int = int(numBatches / 8)
    ema_alpha: float = 0.4
    threshold_mode: str = 'rel'
    threshold: float = 1 / 100
lr_Scheduler_Params_G = Plateua_EMA_Params()

# @dataclass
# class LR_Scheduler_Params_G:
#     gamma: float = 0.5
#     step_size: float = 10 * numBatches
# lr_Scheduler_Params_G = LR_Scheduler_Params_G()

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
gan = MouseGAN(dataset, device, num_feats, num_target_feats, MAX_SEQ_LEN, miniBatchDisc=True, latent_dim=latent_dim,
            g_lr=0.001, d_lr=0.001,
            lr_scheduler=LR_SCHEDULERS.LOSS_GAP_AWARE, schedulerParams=asdict(lr_Scheduler_Params),schedulerParamsG=asdict(lr_Scheduler_Params_G),
            lossFunc=LOSS_FUNC.LSGAN)
if LOAD_PRETRAINED:
    gan.loadPretrained(startingEpoch='save')

gan.train(dataloader, num_epochs, modelSaveInterval=3)
# out.shape:  torch.Size([256, 25, 2]) lstm_out.shape:  torch.Size([256, 25, 512])
# out.shape:  torch.Size([256, 25, 1]) lstm_out.shape:  torch.Size([256, 25, 512])


No initialization for <class 'src.mouseGAN.models.Generator'>
No initialization for <class 'src.mouseGAN.models.Discriminator'>
No initialization for <class 'src.mouseGAN.minibatchDiscrimination.MinibatchDiscrimination'>
	Batch 1/8, d_loss = 0.487, g_loss = 0.646
		 EMA metric: 0.646 metric 0.646
		schedulers step D_lr: 0.0005433809757232666, G_lr: 0.001
	Batch 2/8, d_loss = 0.341, g_loss = 0.491
		 EMA metric: 0.584 metric 0.491
		schedulers step D_lr: 7.704794942796411e-05, G_lr: 0.001
	Batch 3/8, d_loss = 0.293, g_loss = 0.465
		 EMA metric: 0.536 metric 0.465
		schedulers step D_lr: 7.704794942796411e-05, G_lr: 0.001
	Batch 4/8, d_loss = 0.293, g_loss = 0.428
		 EMA metric: 0.493 metric 0.428
		schedulers step D_lr: 7.704795057606803e-06, G_lr: 0.001
	Batch 5/8, d_loss = 0.318, g_loss = 0.335
		 EMA metric: 0.430 metric 0.335
		schedulers step D_lr: 7.704795057606803e-06, G_lr: 0.001
	Batch 6/8, d_loss = 0.372, g_loss = 0.228
		 EMA metric: 0.349 metric 0.228
		schedulers step D_lr

KeyboardInterrupt: 

In [ ]:
gan.save_models('final')

In [ ]:
gan.scheduler_D.scale_history
import pandas as pd

df = pd.DataFrame(gan.scheduler_D.scale_history, columns=['scale'])
df['epoch'] = df.index
df['lr'] = df['scale'].cumprod() * 0.001

fig = go.Figure()

fig.add_trace(go.Scatter(x=df['epoch'], y=df['lr'],))
fig.update_layout(
    title="Learning Rate for discriminator over steps",
    width=800,
    height=800,)
fig.show()

In [ ]:
gan.plotLearningRateScales()

In [ ]:
batch_size = 10
low_radius = 200
high_radius = 300

minTargetWidth = 50
maxTargetWidth = 150
minTargetHeight = 50
maxTargetHeight = 100
TARGET_WIDTH = 150
TARGET_HEIGHT = 100

radiuses = np.random.rand(batch_size) * (high_radius - low_radius) + low_radius
angles = np.random.rand(batch_size) * 2 * np.pi
x = radiuses * np.cos(angles)
y = radiuses * np.sin(angles)
# targetHeights = np.random.rand(batch_size) * (maxTargetHeight - minTargetHeight) + minTargetHeight
targetHeights = np.ones(batch_size) * TARGET_HEIGHT
# targetWidths = np.random.rand(batch_size) * (maxTargetWidth - minTargetWidth) + minTargetWidth
targetWidths = np.ones(batch_size) * TARGET_WIDTH
x_end = np.random.rand(batch_size) * (targetWidths) - targetWidths / 2
y_end = np.random.rand(batch_size) * (targetHeights) - targetHeights / 2
rawButtonTargets = np.stack([targetWidths, targetHeights, x, y, x_end, y_end], axis=1)

norm_rawInput = (rawButtonTargets[:,0:4] - dataset.mean_button) / dataset.std_button
buttonTarget = torch.tensor(norm_rawInput, dtype=torch.float32).to(device)
# buttonTarget = buttonTarget.swapaxes(0,1)

g_states = gan.generator.init_hidden(batch_size)
d_state = gan.discriminator.init_hidden(batch_size)

z = gan.generator.generate_noise(batch_size)
# print(z.shape, buttonTarget.shape, g_states[0][0].shape)
g_feats, _ = gan.generator(z, buttonTarget, g_states)

criterion = nn.MSELoss()

with torch.no_grad():
    rawTrajectories = g_feats * dataset.std_traj + dataset.mean_traj
    g_finalLocations = torch.Tensor(rawButtonTargets[:,2:4]) + rawTrajectories.sum(dim=1)
    targetWidths = torch.Tensor(rawButtonTargets[:,0])
    targetHeights = torch.Tensor(rawButtonTargets[:,1])

    criterion = nn.L1Loss()
    d_loss = criterion(g_finalLocations, torch.Tensor(rawButtonTargets[:,4:6]))


for i in range(len(rawButtonTargets)):
    buttonWidth = rawButtonTargets[i,0]
    buttonHeight = rawButtonTargets[i,1]
    x = rawButtonTargets[i,2]
    dx = rawTrajectories[i,:,0]
    y = rawButtonTargets[i,3]
    dy = rawTrajectories[i,:,1]
    x_end = rawButtonTargets[i,4]
    y_end = rawButtonTargets[i,5]
    print(buttonWidth, buttonHeight, x, y, x_end, y_end)
    # insideBound = insideBounds[i]
    g_finalLocation = g_finalLocations[i]
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=[x], y=[y], mode='markers', name='start'))
    fig.add_trace(go.Scatter(x=[x_end], y=[y_end], mode='markers', name='goal end'))
    fig.add_trace(go.Scatter(x=[g_finalLocation[0]], y=[g_finalLocation[1]], mode='markers', name='generated end'))

    distance = np.sqrt(dx**2 + dy**2)
    velocity = distance / dataset.FIXED_TIMESTEP

    df_sequence = pd.DataFrame(np.stack([dx, dy, velocity], axis=1), columns=['dx', 'dy', 'velocity'])
    df_target = pd.DataFrame([[buttonWidth, buttonHeight, x, y]], columns=['width', 'height', 'start_x', 'start_y'])
    df_abs = dataset.convertToAbsolute(df_sequence, df_target)

    sequence_id = 0
    dataset.SHOW_ONE = True
    fig.add_trace(go.Scatter(x=df_abs['x'], y=df_abs['y'],
            mode='lines+markers',
            marker=dict(
                        size=5, 
                        # symbol= "arrow-bar-up", angleref="previous",
                        # size=15,
                        # color='grey',),
                        color=df_abs['velocity'], colorscale='Viridis', showscale=True, colorbar=dict(title="Velocity")),
            
            ))
    
    x0, y0 = -buttonWidth/2, -buttonHeight/2
    x_i, y_i =  buttonWidth/2, buttonHeight/2
    square = go.layout.Shape(
        type='rect',
        x0=x0,
        y0=y0,
        x1=x_i,
        y1=y_i,
        line=dict(color='black', width=2),
        fillcolor='rgba(0, 0, 255, 0.3)',
    )
    fig.update_layout(
        shapes=[square],
        # title="insideBound: " + str(insideBound),
    )
    fig.show()
    break

# g_feats = g_feats.squeeze(0)
# meanG.append(g_feats.mean(dim=0).cpu().detach().numpy())

# g_feats = g_feats.cpu().detach().numpy()

# input_trajectories, buttonTargets = dataset.denormalize(g_feats, norm_rawInput)


# input_trajectories = []
# buttonTargets = []
# for i in range(len(norm_input_trajectories)):
#     traj = norm_input_trajectories[i] * self.std_traj + self.mean_traj
#     input_trajectories.append(traj)
#     target = norm_buttonTargets[i] * self.std_button + self.mean_button
#     buttonTargets.append(target)
# return input_trajectories, buttonTargets


In [ ]:
data = [[],[],[]]
g_finalLocations = []

points = 100
xlinspace = np.linspace(-TARGET_WIDTH, TARGET_WIDTH, points)
ylinspace = np.linspace(-TARGET_HEIGHT, TARGET_HEIGHT, points)
for x in xlinspace:
    for y in ylinspace:
        g_finalLocations.append([x,y])

batch_size = len(g_finalLocations)
low_radius = 200
high_radius = 300

minTargetWidth = 50
maxTargetWidth = 150
minTargetHeight = 50
maxTargetHeight = 100
TARGET_WIDTH = 150
TARGET_HEIGHT = 100

radiuses = np.random.rand(batch_size) * (high_radius - low_radius) + low_radius
angles = np.random.rand(batch_size) * 2 * np.pi
x = radiuses * np.cos(angles)
y = radiuses * np.sin(angles)
# targetHeights = np.random.rand(batch_size) * (maxTargetHeight - minTargetHeight) + minTargetHeight
targetHeights = np.ones(batch_size) * TARGET_HEIGHT
# targetWidths = np.random.rand(batch_size) * (maxTargetWidth - minTargetWidth) + minTargetWidth
targetWidths = np.ones(batch_size) * TARGET_WIDTH
x_end = np.random.rand(batch_size) * (targetWidths) - targetWidths / 2
y_end = np.random.rand(batch_size) * (targetHeights) - targetHeights / 2
rawButtonTargets = np.stack([targetWidths, targetHeights, x, y, x_end, y_end], axis=1)

targetWidths = torch.Tensor(targetWidths)
targetHeights = torch.Tensor(targetHeights)
# Convert x and y to PyTorch Tensors
g_finalLocations = torch.Tensor(g_finalLocations)

# Coordinates of the button's edges
x_i = -targetWidths / 2
y_i = -targetHeights / 2

# Calculate distances from the point to each edge of the button
dx1 = x_i - g_finalLocations[:, 0]
dx2 = g_finalLocations[:, 0] - (x_i + targetWidths)
dy1 = y_i - g_finalLocations[:, 1]
dy2 = g_finalLocations[:, 1] - (y_i + targetHeights)

# If a distance is negative, the point is inside the button with respect to that edge
insideBounds = (dx1 <= 0) & (dx2 <= 0) & (dy1 <= 0) & (dy2 <= 0)

# Get the maximum distance for x and y (0 if the point is inside the button)
dx = torch.max(dx1, dx2)
dy = torch.max(dy1, dy2)
# Calculate the distances to the nearest corner or edge
dx_dy_gt_0 = (dx > 0) & (dy > 0)  # both dx and dy are > 0, point is outside the button
dists = torch.where(dx_dy_gt_0, torch.sqrt(dx**2 + dy**2), torch.max(dx, dy))  # calculate distance to the corner or edge

# Apply the mask, so that distance is 0 for points inside the button
masked_dists = torch.where(insideBounds, torch.zeros_like(dists), dists)


# Now use these distances in the MSE loss
# g_losses = masked_dists
g_losses = masked_dists ** 0.5

import plotly.graph_objects as go


g_loss_grid = g_losses.reshape(100, 100).detach().numpy() 

fig = go.Figure(data=go.Heatmap(
                   z=g_loss_grid,
                   x=xlinspace,
                   y=ylinspace,
                   colorscale='Viridis'))

rectangle = go.layout.Shape(
    type="rect",
    xref="x",
    yref="y",
    x0=-TARGET_WIDTH/2,
    y0=-TARGET_HEIGHT/2,
    x1=TARGET_WIDTH/2,
    y1=TARGET_HEIGHT/2,
    line=dict(
        color="RoyalBlue",
        width=3,
    ),)


fig.update_layout(
    width = 800,
    height = 800 * TARGET_HEIGHT / TARGET_WIDTH,
    shapes=[rectangle],
    title='Generator Loss depending on the final generated location of the cursor (sqrt(distance))',
    xaxis=dict(title='X'),
    yaxis=dict(title='Y'))

fig.show()



In [20]:

def plotGeneratorSamples():
    fig = go.Figure()
    AXIAL_RESOLUTION = 25
    samples = 100
    rawButtonTargets = dataset.createButtonTargets(samples,
                            low_radius = 100, high_radius = 1000,
                            max_width = 150, min_width = 150,
                            max_height = 100, min_height = 100,
                            axial_resolution = AXIAL_RESOLUTION)
    max_y = np.max(rawButtonTargets[:,3])
    min_y = np.min(rawButtonTargets[:,3])
    max_x = np.max(rawButtonTargets[:,2])
    min_x = np.min(rawButtonTargets[:,2])
    _rawButtonTargets = torch.tensor(rawButtonTargets, dtype=torch.float32).to(device)
    generated_trajs = gan.generate(_rawButtonTargets)
    for i in range(samples):
        generated_traj = generated_trajs[i]
        rawButtonTarget = rawButtonTargets[i]
        df_sequence = pd.DataFrame(generated_traj, columns=dataset.trajColumns)
        df_target = pd.DataFrame([rawButtonTarget], columns=dataset.targetColumns)

        dataset.SHOW_ONE = True

        df_sequence['distance'] = np.sqrt(df_sequence['dx']**2 + df_sequence['dy']**2)
        df_sequence['velocity'] = df_sequence['distance'] / dataset.FIXED_TIMESTEP
        df_abs = dataset.convertToAbsolute(df_sequence, df_target)

        dataset.SHOW_ONE = True
        fig.add_trace(go.Scatter(x=df_abs['x'], y=df_abs['y'],
                mode='lines+markers',
                marker=dict(
                            size=5, 
                            # symbol= "arrow-bar-up", angleref="previous",
                            # size=15,
                            # color='grey',),
                            color=df_abs['velocity'], colorscale='Viridis', showscale=True, colorbar=dict(title="Velocity")),
                
                ))
    
    # x0, y0 = -TARGET_WIDTH/2, -TARGET_HEIGHT/2
    # x_i, y_i =  TARGET_WIDTH/2, TARGET_HEIGHT/2
    # square = go.layout.Shape(
    #     type='rect',
    #     x0=x0,
    #     y0=y0,
    #     x1=x_i,
    #     y1=y_i,
    #     line=dict(color='black', width=2),
    #     fillcolor='rgba(0, 0, 255, 0.3)',
    # )

    fig.update_layout(
        # shapes=[square],
        width=800,
        height=800,
        xaxis=dict(range=[-min_x*1.1, max_x*1.1],),
        yaxis=dict(range=[-min_y*1.1, max_y*1.1],),
    )
    fig.show()

# for epoch in [3,6,'3.5epochs_9000samples_learned_3_directions']:
for epoch in ['final']:
    # gan.loadPretrained(startingEpoch=epoch)
    plotGeneratorSamples()

# for epoch in [10,20,30,40,50]:
#     latest_g_model = find_epoch_model('g', epoch, CKPT_DIR)
#     latest_d_model = find_epoch_model('d', epoch, CKPT_DIR)
#     if latest_g_model is not None:
#         model['g'].load_state_dict(torch.load(latest_g_model))
#         print(f"Loaded generator model: {latest_g_model}")
#     if latest_d_model is not None:
#         model['d'].load_state_dict(torch.load(latest_d_model))
#         print(f"Loaded discriminator model: {latest_d_model}")
#     epoch = min(int(latest_g_model.split('/')[-1].split('.')[0][1:]), int(latest_d_model.split('/')[-1].split('.')[0][1:]))
#     print(f"Starting from epoch {epoch}")
#     plotGeneratorSamples()


In [ ]:
gan.eval()
# z = torch.empty([1, MAX_SEQ_LEN, num_feats]).uniform_().to(device) # random vector
# sampling from spherical distribution
meanG = []
import plotly.graph_objects as go
fig = go.Figure()
for i in range(10):
    for x in range(-100,100, 10):
        z = torch.randn([1, MAX_SEQ_LEN, num_feats]).to(device)
        z = z / z.norm(dim=-1, keepdim=True)

        rawInput = np.array([149.59375,    100.0,       x,      100])
        norm_rawInput = (rawInput - dataset.mean_button) / dataset.std_button
        buttonTarget = torch.tensor([norm_rawInput], dtype=torch.float32).to(device)

        g_states = gan.generator.init_hidden(1)
        d_state = gan.discriminator.init_hidden(1)

        # feed inputs to generator
        g_feats, _ = gan.generator(z, buttonTarget, g_states)
        g_feats = g_feats.squeeze(0)
        # meanG.append(g_feats.mean(dim=0).cpu().detach().numpy())

        # convert back 
        g_feats = g_feats.cpu().detach().numpy()

        input_trajectories, buttonTargets = dataset.denormalize([g_feats], [norm_rawInput])
        input_trajectory = input_trajectories[0]
        buttonTarget = buttonTargets[0]
        df_sequence = pd.DataFrame(input_trajectory, columns=dataset.trajColumns)
        df_target = pd.DataFrame([rawInput], columns=dataset.targetColumns)
        sequence_id = 0
        print("starting location ", rawInput[-2:])
        dataset.SHOW_ONE = True
        # display(df_sequence)
        # display(df_target)
        start_x = rawInput[-2]
        start_y = rawInput[-1]
        sequence_id = 0
        dataset.SHOW_ONE = True

        df_sequence['distance'] = np.sqrt(df_sequence['dx']**2 + df_sequence['dy']**2)
        df_sequence['velocity'] = df_sequence['distance'] / dataset.FIXED_TIMESTEP
        df_abs = dataset.convertToAbsolute(df_sequence, df_target)
        dataset.plotTrajectory(df_abs, df_target[['width','height','start_x','start_y']], sequence_id, fig=fig)